In [1]:
import pandas as pd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString

In [2]:
traj = pd.read_csv('../data/traj.csv')
traj

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays
0,0,2013-10-08T17:45:00Z,254,0,"[116.318726,40.009014]",0.000000,36.6900,0
1,1,2013-10-08T17:46:45Z,254,0,"[116.315102,40.004784]",0.562623,24.5375,0
2,2,2013-10-08T17:47:39Z,254,0,"[116.315018,40.002842]",0.778695,31.9675,0
3,3,2013-10-08T17:49:26Z,254,0,"[116.315041,39.998585]",1.252148,19.7850,0
4,4,2013-10-08T17:51:15Z,254,0,"[116.315605,39.992554]",1.924533,24.4500,0
...,...,...,...,...,...,...,...,...
414839,518403,2013-10-30T23:14:35Z,4954,21998,"[116.333366,39.79073]",21.419211,27.1450,0
414840,518404,2013-10-30T23:14:59Z,4954,21998,"[116.330444,39.79084]",21.669183,30.0825,0
414841,518405,2013-10-30T23:16:35Z,4954,21998,"[116.328453,39.793888]",22.129037,46.0550,0
414842,518406,2013-10-30T23:16:59Z,4954,21998,"[116.328072,39.796574]",22.429494,51.5150,0


In [9]:
traj['time'] = pd.to_datetime(traj['time'])
traj['time_diff'] = traj['time'].diff()
traj['dis_diff'] = traj['current_dis'].diff() * 1000
traj['cost_time'] = traj['time_diff'].dt.total_seconds() / 60
traj['time_period'] = traj['time'].apply(lambda x: x.hour)
traj

,id,time,entity_id,traj_id,coordinates,current_dis,speeds,holidays,time_diff,dis_diff,cost_time,time_period
0,0,2013-10-08 17:45:00+00:00,254,0,"[116.318726,40.009014]",0.000000,36.6900,0,NaT,NaN,NaN,17
1,1,2013-10-08 17:46:45+00:00,254,0,"[116.315102,40.004784]",0.562623,24.5375,0,0 days 00:01:45,562.623059,1.750000,17
2,2,2013-10-08 17:47:39+00:00,254,0,"[116.315018,40.002842]",0.778695,31.9675,0,0 days 00:00:54,216.072372,0.900000,17
3,3,2013-10-08 17:49:26+00:00,254,0,"[116.315041,39.998585]",1.252148,19.7850,0,0 days 00:01:47,473.452423,1.783333,17
4,4,2013-10-08 17:51:15+00:00,254,0,"[116.315605,39.992554]",1.924533,24.4500,0,0 days 00:01:49,672.385605,1.816667,17
...,...,...,...,...,...,...,...,...,...,...,...,...
414839,518403,2013-10-30 23:14:35+00:00,4954,21998,"[116.333366,39.79073]",21.419211,27.1450,0,0 days 00:00:36,209.337371,0.600000,23
414840,518404,2013-10-30 23:14:59+00:00,4954,21998,"[116.330444,39.79084]",21.669183,30.0825,0,0 days 00:00:24,249.972523,0.400000,23
414841,518405,2013-10-30 23:16:35+00:00,4954,21998,"[116.328453,39.793888]",22.129037,46.0550,0,0 days 00:01:36,459.854339,1.600000,23
414842,518406,2013-10-30 23:16:59+00:00,4954,21998,"[116.328072,39.796574]",22.429494,51.5150,0,0 days 00:00:24,300.456659,0.400000,23


In [17]:
from tqdm import *
df = pd.DataFrame(columns=['traj_id','geom', 'coordinate1', 'coordinate2', 'cost_time', 'dis_diff', 'holidays','time_period'])

trajs = traj.groupby('traj_id')
for name, group in tqdm(trajs):
    group = group.reset_index(drop=True)
    if(len(group) > 1):
        for i in range(len(group)-1):
            geom = LineString([tuple(eval(group.loc[i,'coordinates'])), tuple(eval(group.loc[i+1,'coordinates']))])
            new_row = pd.DataFrame({
            'traj_id':[name], 
            'geom': [geom], 
            'coordinate1': [tuple(eval(group.loc[i,'coordinates']))],
            'coordinate2': [tuple(eval(group.loc[i+1,'coordinates']))],
            'cost_time': [group.loc[i+1,'cost_time']],
            'dis_diff': [group.loc[i+1,'dis_diff']],
            'holidays': [group.loc[i+1,'holidays']],
            'time_period': [group.loc[i+1,'time_period']]
            })
            df = pd.concat([df,new_row])
df


  0%|          | 0/17600 [00:00<?, ?it/s]C:\Users\Kafka\AppData\Local\Temp\ipykernel_1140\3115936679.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,new_row])
100%|██████████| 17600/17600 [2:13:24<00:00,  2.20it/s]  


,traj_id,geom,coordinate1,coordinate2,cost_time,dis_diff,holidays,time_period
0,0,"LINESTRING (116.318726 40.009014, 116.315102 4...","(116.318726, 40.009014)","(116.315102, 40.004784)",1.750000,562.623059,0,17
0,0,"LINESTRING (116.315102 40.004784, 116.315018 4...","(116.315102, 40.004784)","(116.315018, 40.002842)",0.900000,216.072372,0,17
0,0,"LINESTRING (116.315018 40.002842, 116.315041 3...","(116.315018, 40.002842)","(116.315041, 39.998585)",1.783333,473.452423,0,17
0,0,"LINESTRING (116.315041 39.998585, 116.315605 3...","(116.315041, 39.998585)","(116.315605, 39.992554)",1.816667,672.385605,0,17
0,0,"LINESTRING (116.315605 39.992554, 116.315735 3...","(116.315605, 39.992554)","(116.315735, 39.987846)",0.900000,523.655150,0,17
...,...,...,...,...,...,...,...,...
0,21998,"LINESTRING (116.335815 39.79068, 116.333366 39...","(116.335815, 39.79068)","(116.333366, 39.79073)",0.600000,209.337371,0,23
0,21998,"LINESTRING (116.333366 39.79073, 116.330444 39...","(116.333366, 39.79073)","(116.330444, 39.79084)",0.400000,249.972523,0,23
0,21998,"LINESTRING (116.330444 39.79084, 116.328453 39...","(116.330444, 39.79084)","(116.328453, 39.793888)",1.600000,459.854339,0,23
0,21998,"LINESTRING (116.328453 39.793888, 116.328072 3...","(116.328453, 39.793888)","(116.328072, 39.796574)",0.400000,300.456659,0,23


In [19]:
df = df.reset_index(drop=True)
df

,traj_id,geom,coordinate1,coordinate2,cost_time,dis_diff,holidays,time_period
0,0,"LINESTRING (116.318726 40.009014, 116.315102 4...","(116.318726, 40.009014)","(116.315102, 40.004784)",1.750000,562.623059,0,17
1,0,"LINESTRING (116.315102 40.004784, 116.315018 4...","(116.315102, 40.004784)","(116.315018, 40.002842)",0.900000,216.072372,0,17
2,0,"LINESTRING (116.315018 40.002842, 116.315041 3...","(116.315018, 40.002842)","(116.315041, 39.998585)",1.783333,473.452423,0,17
3,0,"LINESTRING (116.315041 39.998585, 116.315605 3...","(116.315041, 39.998585)","(116.315605, 39.992554)",1.816667,672.385605,0,17
4,0,"LINESTRING (116.315605 39.992554, 116.315735 3...","(116.315605, 39.992554)","(116.315735, 39.987846)",0.900000,523.655150,0,17
...,...,...,...,...,...,...,...,...
397239,21998,"LINESTRING (116.335815 39.79068, 116.333366 39...","(116.335815, 39.79068)","(116.333366, 39.79073)",0.600000,209.337371,0,23
397240,21998,"LINESTRING (116.333366 39.79073, 116.330444 39...","(116.333366, 39.79073)","(116.330444, 39.79084)",0.400000,249.972523,0,23
397241,21998,"LINESTRING (116.330444 39.79084, 116.328453 39...","(116.330444, 39.79084)","(116.328453, 39.793888)",1.600000,459.854339,0,23
397242,21998,"LINESTRING (116.328453 39.793888, 116.328072 3...","(116.328453, 39.793888)","(116.328072, 39.796574)",0.400000,300.456659,0,23


In [20]:
df.to_csv('traj_pair_infor.csv')

In [40]:
from tqdm import *
df = pd.DataFrame(columns=['traj_id','geom'])

trajs = traj.groupby('traj_id')
for name, group in tqdm(trajs):
    # print(len(group))
    # print(group)
    series = pd.Series(group['coordinates'])
    series = series.reset_index(drop=True)
    if(len(series) > 1):
        for i in range(len(series)-1):
            line = LineString([tuple(eval(series[i])), tuple(eval(series[i+1]))])
            new_row = pd.DataFrame({'traj_id':[name], 'geom': [line]})
            df = pd.concat([df,new_row])
df


100%|██████████| 17600/17600 [48:00<00:00,  6.11it/s]  


,traj_id,geom
0,0,"LINESTRING (116.318726 40.009014, 116.315102 4..."
0,0,"LINESTRING (116.315102 40.004784, 116.315018 4..."
0,0,"LINESTRING (116.315018 40.002842, 116.315041 3..."
0,0,"LINESTRING (116.315041 39.998585, 116.315605 3..."
0,0,"LINESTRING (116.315605 39.992554, 116.315735 3..."
...,...,...
0,21998,"LINESTRING (116.335815 39.79068, 116.333366 39..."
0,21998,"LINESTRING (116.333366 39.79073, 116.330444 39..."
0,21998,"LINESTRING (116.330444 39.79084, 116.328453 39..."
0,21998,"LINESTRING (116.328453 39.793888, 116.328072 3..."


In [44]:
df = df.reset_index(drop=True)
df

,traj_id,geom
0,0,"LINESTRING (116.318726 40.009014, 116.315102 4..."
1,0,"LINESTRING (116.315102 40.004784, 116.315018 4..."
2,0,"LINESTRING (116.315018 40.002842, 116.315041 3..."
3,0,"LINESTRING (116.315041 39.998585, 116.315605 3..."
4,0,"LINESTRING (116.315605 39.992554, 116.315735 3..."
...,...,...
397239,21998,"LINESTRING (116.335815 39.79068, 116.333366 39..."
397240,21998,"LINESTRING (116.333366 39.79073, 116.330444 39..."
397241,21998,"LINESTRING (116.330444 39.79084, 116.328453 39..."
397242,21998,"LINESTRING (116.328453 39.793888, 116.328072 3..."


In [45]:
df.to_csv('traj_pair.csv')